In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from statsmodels.stats.multitest import multipletests

In [19]:
def pca(X, D):
    """
    PCA
    Input:
        X - An (1024, 1024) numpy array
        D - An int less than 1024. The target dimension
    Output:
        X - A compressed (1024, 1024) numpy array
    """
    X = (X - np.mean(X, axis=0))/np.std(X, axis=0)
    u, e, v = np.linalg.svd(np.dot(X.T, X))
    eigen = v.T[:, 0:D]
    final = np.dot(X, eigen)
    #return np.dot(final, eigen.T)
    return final

In [20]:
def sklearn_pca(X, D):
    """
    Your PCA implementation should be equivalent to this function.
    Do not use this function in your implementation!
    """
    X = (X - np.mean(X, axis=0))/np.std(X, axis=0)
    from sklearn.decomposition import PCA
    p = PCA(n_components=D, svd_solver='full')
    trans_pca = p.fit_transform(X)
    var = p.explained_variance_ratio_
    return trans_pca, var

In [21]:
X = pd.read_csv("Data/BAL/normdataBAL0715.txt", sep = '\t', usecols=range(2,156))
df = X

In [22]:
anova_results = pd.DataFrame(index=df.index, columns=['F-value', 'p-value'])

for gene in df.index:
    group1 = df.loc[gene, df.columns.str.contains('NC')]
    group2 = df.loc[gene, df.columns.str.contains('SA')]
    group3 = df.loc[gene, df.columns.str.contains('notSA')]
    group4 = df.loc[gene, df.columns.str.contains('VSA')]
    f_value, p_value = stats.f_oneway(group1, group2, group3, group4)
    anova_results.loc[gene, 'F-value'] = f_value
    anova_results.loc[gene, 'p-value'] = p_value

In [23]:
alpha = 0.05
bh_corrected_pvals = multipletests(anova_results['p-value'], method='fdr_bh')[1]
filtered_genes = anova_results.index[bh_corrected_pvals < alpha]

In [25]:
filtered_df = df.loc[filtered_genes]
filtered_df.to_csv("filtered_data_BAL.csv")

In [ ]:

D = 2
pca_final=pca(X, D)
pca_sklearn, var=sklearn_pca(X, D)
print(var)

plt.scatter(pca_sklearn[:, 0], pca_sklearn[:, 1])
plt.title('PCA')
plt.show()

In [ ]:
# #Create heatmap
# sns.clustermap(df_gene, cmap='coolwarm', center=0, metric='euclidean',
#                row_cluster=True, figsize=(20, 10), dendrogram_ratio=(0.1, 0.2),
#                cbar_kws={'label': 'Expression for BAL'},
#                yticklabels=False, xticklabels=True)